<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/4_OtimizacaoDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.storagelevel import StorageLevel

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import files
files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
produtos = spark.read.csv("produtos.csv", header=True, inferSchema=True)
vendedores = spark.read.csv("vendedores.csv", header=True, inferSchema=True)
clientes = spark.read.csv("clientes.csv", header=True, inferSchema=True)
itens_pedido = spark.read.csv("itens_pedido.csv", header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv("pagamentos_pedido.csv", header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv("avaliacoes_pedido.csv", header=True, inferSchema=True)
pedidos = spark.read.csv("pedidos.csv", header=True, inferSchema=True)

In [ ]:
clientes.show()
pedidos.show()
itens_pedido.show()
pagamentos_pedido.show()

In [ ]:
clientes.createOrReplaceTempView("clientes")
spark.sql('CACHE TABLE clientes')
spark.sql("SELECT COUNT(*) FROM clientes").show()
spark.sql('UNCACHE TABLE clientes')


In [ ]:
print(itens_pedido.count())
print(pedidos.count())

small_pedidos_df = broadcast(pedidos)

boradcast_data = itens_pedido.join(small_pedidos_df, on='id_pedido')
boradcast_data.show()

In [ ]:
salario_medio_valor = pagamentos_pedido.agg(avg('valor_pagamento').alias('pagamemto_medio')).collect()[0][0]
pedidos_filtrados_df = pagamentos_pedido.filter(pagamentos_pedido['valor_pagamento'] > salario_medio_valor)

salario_medio_df = pagamentos_pedido.select(avg("valor_pagamento").alias("pagamento_medio"))
pedidos_filtrados_join_df = pagamentos_pedido.join(
    salario_medio_df,
    pagamentos_pedido["valor_pagamento"] > salario_medio_df["pagamento_medio"]
).select(pagamentos_pedido["*"])
salario_medio_df.show()
pedidos_filtrados_join_df.show()

In [ ]:
start_time = time.time()

salario_medio_valor = pagamentos_pedido.agg(avg('valor_pagamento').alias('pagamemto_medio')).collect()[0][0]
pedidos_filtrados_df = pagamentos_pedido.filter(pagamentos_pedido['valor_pagamento'] > salario_medio_valor)

end_time = time.time()
print(end_time - start_time)

start_time = time.time()
salario_medio_df = pagamentos_pedido.select(avg("valor_pagamento").alias("pagamento_medio"))
pedidos_filtrados_join_df = pagamentos_pedido.join(
    salario_medio_df,
    pagamentos_pedido["valor_pagamento"] > salario_medio_df["pagamento_medio"]
).select(pagamentos_pedido["*"])

end_time = time.time()
print(end_time - start_time)

In [ ]:
spark.stop()